In [14]:
import pandas as pd
import numpy as np
import os

DATA_PATH = os.path.join(os.path.dirname(os.getcwd()), "data")

df = pd.read_excel(os.path.join(DATA_PATH, "Nordic_Textile_Anatomy_Database_DdS.xlsx"), sheet_name="RMM_DK")


In [15]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def cluster_composition_by_category(df, fiber_cols=3, min_clusters=2, max_clusters=4):
    """
    Clusters textiles by fiber composition within each Category.
    
    Parameters:
    - df: DataFrame with columns:
        'Category',
        'Fibre 1', 'Fibre 1 % Range', ..., up to 'Fibre {fiber_cols}', 'Fibre {fiber_cols} % Range'
    - fiber_cols: number of fiber columns to consider (default 3)
    - min_clusters, max_clusters: range of k to try for KMeans
    
    Returns:
    - dict mapping category to summary DataFrame with columns:
        'Cluster', 'Count', and one column per fiber name giving average percentage in that cluster.
    """
    # Helper to parse percentage range to midpoint
    def parse_pct(s):
        try:
            s = str(s).replace('–', '-').replace('%', '')
            low, high = s.split('-')
            return (float(low) + float(high)) / 2
        except:
            return np.nan

    # Collect all unique fiber names from columns Fibre 1 .. Fibre fiber_cols
    fiber_names = set()
    parsed_entries = []
    for idx, row in df.iterrows():
        comp = {}
        for i in range(1, fiber_cols + 1):
            name = row.get(f'Fibre {i}')
            pct_range = row.get(f'Fibre {i} % Range')
            if pd.notna(name) and pd.notna(pct_range):
                pct = parse_pct(pct_range)
                if pd.notna(pct):
                    comp[name] = pct
                    fiber_names.add(name)
        parsed_entries.append(comp)

    all_fibers = sorted(fiber_names)
    if not all_fibers:
        print("No fiber data found. Check column names and data.")
        return {}

    # Build feature matrix: each row is normalized composition vector over all_fibers
    feature_rows = []
    indices = []
    categories = []
    for (idx, row), comp in zip(df.iterrows(), parsed_entries):
        if comp:
            vec = [comp.get(f, 0.0) for f in all_fibers]
            total = sum(vec)
            if total > 0:
                vec = [v / total for v in vec]
                feature_rows.append(vec)
                indices.append(idx)
                categories.append(row['Category'])
    if not feature_rows:
        print("No valid composition entries to cluster.")
        return {}

    feat_df = pd.DataFrame(feature_rows, index=indices, columns=all_fibers)
    result = {}

    # Group by category
    cat_series = pd.Series(categories, index=indices, name='Category')
    for cat, group in cat_series.groupby(cat_series):
        idxs = group.index
        X = feat_df.loc[idxs]
        n_samples = len(X)
        if n_samples < 2:
            print(f"Category '{cat}' has fewer than 2 samples, skipping.")
            continue

        # Determine best k by silhouette
        best_k = None
        best_score = -1
        for k in range(min_clusters, min(max_clusters, n_samples - 1) + 1):
            km = KMeans(n_clusters=k, random_state=0)
            labels = km.fit_predict(X)
            # Silhouette requires at least 2 clusters and less than n_samples clusters
            score = silhouette_score(X, labels)
            if score > best_score:
                best_score = score
                best_k = k

        km = KMeans(n_clusters=best_k, random_state=0).fit(X)
        labels = km.labels_
        centroids = km.cluster_centers_

        # Build summary for this category
        summary = []
        for cluster_label in range(best_k):
            mask = labels == cluster_label
            count = int(mask.sum())
            centroid = centroids[cluster_label]
            # Convert centroid to percentages summing to 100
            pct = centroid / centroid.sum() * 100
            comp_dict = {f: pct_val for f, pct_val in zip(all_fibers, pct)}
            row_summary = {'Cluster': cluster_label, 'Count': count}
            row_summary.update(comp_dict)
            summary.append(row_summary)

        summary_df = pd.DataFrame(summary).sort_values('Cluster').reset_index(drop=True)

        # drop insignificant fibers
        summary_df = summary_df.loc[:, summary_df.max() >= 1]
        cols_to_clean = summary_df.select_dtypes(include='number').columns

        # Replace values < 1 with 0
        summary_df[cols_to_clean] = summary_df[cols_to_clean].where(summary_df[cols_to_clean] >= 1, 0)

        # check to what percentage fibres add up
        meta_cols = ['Cluster', 'Count']
        fiber_cols = [col for col in summary_df.columns if col not in meta_cols]
        summary_df['Sum'] = summary_df[fiber_cols].sum(axis=1)
        summary_df = summary_df[['Sum'] + meta_cols + fiber_cols]

        result[cat] = summary_df

    return result

# Run clustering if df exists
if 'df' not in globals():
    print("Please ensure your DataFrame is named 'df' with columns 'Category', "
          "'Fibre 1'..'Fibre 3', 'Fibre 1 % Range'..'Fibre 3 % Range'.")
else:
    clusters = cluster_composition_by_category(df)
    for cat, summary_df in clusters.items():
        print(f"\nCategory: {cat}")
        display(summary_df)



Category: Dresses and skirts


,Sum,Cluster,Count,Acetate,Acrylic,Cotton,Cupro,Flax/linen,Lyocell,Modal,Polyamide/nylon,Polyester,Silk,True Hemp,Viscose,Wool
0,98.843630,0,101,0.000000,0.000000,1.195961,1.088129,1.568284,0.00000,0.000000,8.361745,3.498123,0.000000,0.000000,83.131388,0.000000
1,97.172286,1,144,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,93.560133,0.000000,0.000000,3.612153,0.000000
2,97.517588,2,119,0.000000,0.000000,90.798780,0.000000,0.000000,0.00000,0.000000,1.626029,5.092779,0.000000,0.000000,0.000000,0.000000
3,99.229601,3,48,2.083333,2.550758,2.574303,0.000000,8.215603,17.81778,20.694699,19.087874,12.560447,8.333333,1.144802,0.000000,4.166667



Category: Handkerchiefs, ties, scarves, gloves and other


,Sum,Cluster,Count,Acrylic,"Animal hair (alpaca, llama, camel, kashmir goat, angora goat, angora rabbit)",Cotton,Flax/linen,Other,Polyacrylate,Polyamide/nylon,Polyester,Polyurethane,Silk,Viscose,Wool
0,96.997597,0,211,2.931342,0.000000,2.183223,0.000000,0.000000,0.000000,1.983351,89.899681,0.000000,0.0000,0.000000,0.000000
1,98.680669,1,115,0.000000,0.000000,96.314634,0.000000,0.000000,0.000000,0.000000,2.366035,0.000000,0.0000,0.000000,0.000000
2,96.687029,2,179,19.799183,4.932344,0.000000,1.303538,1.740917,1.033936,17.714091,4.681400,1.799686,10.2426,1.829519,31.609814



Category: Overcoats and Anoraks


...

,Sum,Cluster,Count,Cotton,Elastane/Spandex,Flax/linen,Lyocell,Modal,Other,Polyamide/nylon,Polyester,Viscose,Wool
0,96.497790,0,171,89.954092,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,6.543698,0.000000,0.000000
1,99.124346,1,88,3.845659,0.00000,4.171246,4.882988,1.873312,1.136364,0.000000,64.637184,15.552572,3.025021
2,98.315382,2,34,0.000000,2.17213,0.000000,0.000000,0.000000,0.000000,84.154528,2.032533,9.956191,0.000000
3,100.000000,3,7,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000



Category: Trousers and shorts


,Sum,Cluster,Count,"Animal hair (alpaca, llama, camel, kashmir goat, angora goat, angora rabbit)",Cotton,Cupro,Elastane/spandex,Flax/linen,Modal,Polyamide/nylon,Polyester,Polyurethane,Silk,Viscose,Wool
0,99.530516,0,51,1.271598,2.100544,1.077461,1.417717,2.614379,3.813516,32.121806,6.459200,1.960784,1.960784,30.984152,13.748575
1,97.198695,1,85,0.000000,9.046691,0.000000,0.000000,0.000000,0.000000,0.000000,80.871812,0.000000,0.000000,7.280191,0.000000
2,96.678765,2,158,0.000000,90.219034,0.000000,0.000000,0.000000,0.000000,0.000000,6.459731,0.000000,0.000000,0.000000,0.000000



Category: Underwear, socks, night clothes


,Sum,Cluster,Count,Acrylic,Cotton,Elastane/Spandex,Elastane/spandex,Flax/linen,Lyocell,Modal,Other,Polyamide/nylon,Polyester,Polypropylene,Viscose,Wool
0,96.980018,0,130,0.000000,0.000000,3.169775,5.734864,0.000000,0.000000,0.000000,0.000000,88.075378,0.000000,0.000000,0.000000,0.000000
1,98.989802,1,214,0.000000,83.363659,0.000000,0.000000,0.000000,0.000000,0.000000,1.263151,7.841908,6.521084,0.000000,0.000000,0.000000
2,95.157020,2,84,0.000000,1.731712,0.000000,0.000000,1.190476,2.380952,3.571429,0.000000,5.973528,56.941145,0.000000,23.367777,0.000000
3,99.123504,3,51,9.498652,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.675343,0.000000,3.444478,0.000000,64.505031


In [16]:
merged = []

for category, df_cat in clusters.items():
    df_cat = df_cat.copy()
    df_cat['Category'] = category  # Add category column
    merged.append(df_cat)

# Combine all into one DataFrame
merged_df = pd.concat(merged, ignore_index=True)

# Sort by Category (A-Z), then Count (descending)
merged_df = merged_df.sort_values(by=['Category', 'Count'], ascending=[True, False]).reset_index(drop=True)

cols = ['Category'] + [col for col in merged_df.columns if col != 'Category']
merged_df = merged_df[cols]
merged_df


,Category,Sum,Cluster,Count,Acetate,Acrylic,Cotton,Cupro,Flax/linen,Lyocell,...,Wool,"Animal hair (alpaca, llama, camel, kashmir goat, angora goat, angora rabbit)",Other,Polyacrylate,Polyurethane,Polyimide,Elastane/Spandex,Elastane/spandex,Polyethylene,Polypropylene
0,Dresses and skirts,97.172286,1,144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dresses and skirts,97.517588,2,119,0.000000,0.000000,90.798780,0.000000,0.000000,0.000000,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dresses and skirts,98.843630,0,101,0.000000,0.000000,1.195961,1.088129,1.568284,0.000000,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dresses and skirts,99.229601,3,48,2.083333,2.550758,2.574303,0.000000,8.215603,17.817780,...,4.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Handkerchiefs, ties, scarves, gloves and other",96.997597,0,211,NaN,2.931342,2.183223,NaN,0.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
5,"Handkerchiefs, ties, scarves, gloves and other",96.687029,2,179,NaN,19.799183,0.000000,NaN,1.303538,NaN,...,31.609814,4.932344,1.740917,1.033936,1.799686,NaN,NaN,NaN,NaN,NaN
6,"Handkerchiefs, ties, scarves, gloves and other",98.680669,1,115,NaN,0.000000,96.314634,NaN,0.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
7,Overcoats and Anoraks,97.385936,0,279,NaN,0.000000,2.802936,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.114305,0.000000,NaN,NaN,NaN,NaN
8,Overcoats and Anoraks,98.332154,2,136,NaN,0.000000,92.495112,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
9,Overcoats and Anoraks,98.462229,1,100,NaN,0.000000,1.455446,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN


In [19]:
merged_df.sort_values(by="Count", ascending=False)

,Category,Sum,Cluster,Count,Acetate,Acrylic,Cotton,Cupro,Flax/linen,Lyocell,...,Wool,"Animal hair (alpaca, llama, camel, kashmir goat, angora goat, angora rabbit)",Other,Polyacrylate,Polyurethane,Polyimide,Elastane/Spandex,Elastane/spandex,Polyethylene,Polypropylene
30,"T-shirts, singlets and vests, hoodies and crew...",99.161190,0,417,NaN,0.000000,92.746798,NaN,0.000000,0.000000,...,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Overcoats and Anoraks,97.385936,0,279,NaN,0.000000,2.802936,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.114305,0.000000,NaN,NaN,NaN,NaN
41,"Underwear, socks, night clothes",98.989802,1,214,NaN,0.000000,83.363659,NaN,0.000000,0.000000,...,0.000000,NaN,1.263151,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000
4,"Handkerchiefs, ties, scarves, gloves and other",96.997597,0,211,NaN,2.931342,2.183223,NaN,0.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
11,"Shirts, Blouses, Tops",98.408673,1,196,NaN,0.000000,95.470405,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
15,Sportswear and swimwear,99.272741,0,183,NaN,NaN,0.000000,NaN,NaN,0.000000,...,0.000000,NaN,0.000000,NaN,NaN,NaN,2.976402,3.797684,0.000000,NaN
5,"Handkerchiefs, ties, scarves, gloves and other",96.687029,2,179,NaN,19.799183,0.000000,NaN,1.303538,NaN,...,31.609814,4.932344,1.740917,1.033936,1.799686,NaN,NaN,NaN,NaN,NaN
12,"Shirts, Blouses, Tops",97.189172,0,176,NaN,0.000000,2.639496,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
34,Trousers and Shorts,96.497790,0,171,NaN,NaN,89.954092,NaN,0.000000,0.000000,...,0.000000,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
38,Trousers and shorts,96.678765,2,158,NaN,NaN,90.219034,0.000000,0.000000,NaN,...,0.000000,0.000000,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN


In [21]:
# Select top 10 rows
top10 = merged_df.head(10).copy()

# Identify fibre columns (exclude meta columns)
meta_cols = ['Category', 'Sum', 'Cluster', 'Count']
fiber_cols = [col for col in top10.columns if col not in meta_cols]

# Filter out fibre columns where the max value is less than 5%
keep_fibers = [col for col in fiber_cols if top10[col].max(skipna=True) >= 5]

# Keep only meta columns and filtered fibre columns
filtered_top10 = top10[meta_cols + keep_fibers]
filtered_top10 = filtered_top10.drop(columns=['Cluster'])
filtered_top10

,Category,Sum,Count,Acrylic,Cotton,Polyamide/nylon,Polyester,Silk,Wool
30,"T-shirts, singlets and vests, hoodies and crew...",99.161190,417,0.000000,92.746798,0.000000,5.203479,NaN,0.000000
7,Overcoats and Anoraks,97.385936,279,0.000000,2.802936,0.000000,93.468695,NaN,0.000000
41,"Underwear, socks, night clothes",98.989802,214,0.000000,83.363659,7.841908,6.521084,NaN,0.000000
4,"Handkerchiefs, ties, scarves, gloves and other",96.997597,211,2.931342,2.183223,1.983351,89.899681,0.0000,0.000000
11,"Shirts, Blouses, Tops",98.408673,196,0.000000,95.470405,1.365503,1.572766,0.0000,0.000000
15,Sportswear and swimwear,99.272741,183,NaN,0.000000,1.091954,91.406700,NaN,0.000000
5,"Handkerchiefs, ties, scarves, gloves and other",96.687029,179,19.799183,0.000000,17.714091,4.681400,10.2426,31.609814
12,"Shirts, Blouses, Tops",97.189172,176,0.000000,2.639496,1.380967,93.168709,0.0000,0.000000
34,Trousers and Shorts,96.497790,171,NaN,89.954092,0.000000,6.543698,NaN,0.000000
38,Trousers and shorts,96.678765,158,NaN,90.219034,0.000000,6.459731,0.0000,0.000000


In [22]:
# Export merged_df as an Excel file
output_path = os.path.join(DATA_PATH, "clustered_fiber_composition.xlsx")
merged_df.to_excel(output_path, index=False)